In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import os
os.environ['pRT_input_data_path'] = os.path.join('/home/mvasist/pRT/input_data_paper/input_data')

from petitRADTRANS.retrieval.rebin_give_width import rebin_give_width
from petitRADTRANS.retrieval.data import Data
from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc



## Rebinning to pRT wavelengths individually in each channel

iesfkdsnf

In [8]:
'''
Generate wavelengths of resolution 1000 in pRT and create and store in wavelengths folder. 
'''

'\nGenerate wavelengths of resolution 1000 in pRT and create and store in wavelengths folder. \n'

In [9]:
def str_to_numpy(lst):
    return [float(i) for i in lst]

In [10]:
def wlenbins(w1):
    wlen_bins = np.zeros_like(w1)
    wlen_bins[:-1] = np.diff(w1)
    wlen_bins[-1] = wlen_bins[-2]
    return wlen_bins

def simulating_wavelengths(a,b):
    
    pRT_object = Radtrans(['CH4_hargreaves',
#                         'H2O_Exomol',
#                         'CO2',
#                         'CO_all_iso_HITEMP',
#                         'H2S',
#                         'NH3',
#                         'PH3',
#                         'Na_allard',
#                         'K_allard',
#                         '15NH3' ,
                        'SO2'],
                     continuum_opacities=['H2-H2', 'H2-He'],
                     rayleigh_species=['He', 'H2'],
                     wlen_bords_micron=[a,b])  
    
    wlen = nc.c/pRT_object.freq/1e-4
    
    return wlen

def flux_rebin(spec, ch):
    a = np.arange(0, len(spec))
    ind = a[spec['Band'] == ch]
    print(ch)
    w , fl, fl_err = np.array(spec.iloc[ind, 0]), np.array(spec.iloc[ind,1]), np.array(spec.iloc[ind,2])
    mirisim_wlen_ch = simulating_wavelengths(w[0]+0.009, w[-1]-0.009)
    wlen_bins = wlenbins(mirisim_wlen_ch)
#     print(wlen_bins)
    xx = Data.convolve(w, fl, 1000)
    xx_error = Data.convolve(w, fl_err, 1000)
    flux_rebinned = torch.from_numpy(rebin_give_width(w, xx, mirisim_wlen_ch, wlen_bins))
    flux_rebinned_err = torch.from_numpy(rebin_give_width(w, xx_error, mirisim_wlen_ch, wlen_bins))
    return torch.stack((torch.from_numpy(mirisim_wlen_ch), flux_rebinned, flux_rebinned_err), dim=1)


In [13]:
for p in ['WISE0855_spectrum_180423_tillwl18.txt']: #'ROSS458C_MIRI_260623_tillwl18.txt', 'WISE0458_spectrum_221122_tillwl18.txt','WISE0855_spectrum_180423_tillwl18.txt'
    path = Path('.') / p
    spec= pd.DataFrame()
    spec = pd.read_csv(path, header=0, delimiter = ',')
    
     # converting to numpy 
    for i in range(3):
        if type(spec.iloc[:,i].values[0]) == str:
            spec.iloc[:,i] = str_to_numpy(spec.iloc[:,i])
            
    channels = ['1A', '1B', '1C', '2A', '2B', '2C','3A', '3B', '3C','4A']
    
    flux_rebinned = torch.cat([flux_rebin(spec, ch) for ch in channels])
    print(len(spec.iloc[:,0]), flux_rebinned.size())
    rebinned_spectrum_pd = pd.DataFrame(flux_rebinned.numpy())
    col_list = ['wavelength[um]', 'flux[Jy]', 'error[Jy]']
    rebinned_spectrum_pd.columns = col_list
    rebinned_spectrum_pd.to_csv('rebinned_observation_2_' + p.split('_')[0] +'.csv', sep=',', index=False)
    


1A


/home/mvasist/miniconda3/envs/WISEJ1828/lib/python3.9/site-packages/petitRADTRANS/radtrans.py:113: FutureWarning: pRT_input_data_path was set by an environment variable. In a future update, the path to the petitRADTRANS input_data will be set within a .ini file that will be automatically generated into the user home directory (OS agnostic), inside a .petitradtrans directory
  warnings.warn(f"pRT_input_data_path was set by an environment variable. In a future update, the path to "


  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

1B
  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

1C
  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

2A
  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

2B
  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.

2C
  Read line opacities of CH4_hargreaves...
 Done.
  Read line opacities of SO2...
 Done.

  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Do

In [14]:
spec

,wavelength[um],flux[Jy],error[Jy],Band
0,4.900400,0.000006,0.000015,1A
1,4.901201,0.000070,0.000017,1A
2,4.902002,0.000207,0.000019,1A
3,4.902802,0.000324,0.000018,1A
4,4.903603,0.000604,0.000016,1A
...,...,...,...,...
9782,17.973500,0.002079,0.000046,4A
9783,17.979511,0.001978,0.000078,4A
9784,17.985522,0.001827,0.000077,4A
9785,17.991533,0.001895,0.000079,4A


In [63]:
'''
The spec (obs/sim) that is being 
rebinned should always be longer than the simulation such that 
- spec[1] <= ref_spec[1] - wl_bins[1]/2
- spec[-1] >= ref_spec[-1] + wl_bins[1]/2
Otherwise, it doesn't work and throw the error - 
STOP rebin.f90 ERROR: extent of fine wavelength grid is too small! - 

'''

(9746, torch.Size([1383, 3]))

In [59]:
flux_rebinned

tensor([[4.9147e+00, 3.6194e-04, 2.4510e-05],
        [4.9196e+00, 3.4031e-04, 2.5004e-05],
        [4.9245e+00, 4.2764e-04, 2.7036e-05],
        ...,
        [1.7925e+01, 1.0080e-04, 5.4289e-05],
        [1.7943e+01, 1.1931e-04, 5.2920e-05],
        [1.7961e+01, 1.9651e-04, 5.3332e-05]], dtype=torch.float64)